# Quiz 6

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [2]:
data = pd.read_csv('homework_6.1.csv')

In [3]:
data.head()

,Z,X,Y
0,0.548814,0,-0.823220
1,0.715189,1,0.842405
2,0.602763,1,0.898618
3,0.544883,0,-0.817325
4,0.423655,0,-0.635482


In [5]:
# Split data
treated = data[data["X"] == 1]
control = data[data["X"] == 0]

X_treated = treated[["Z"]].values
X_control = control[["Z"]].values

In [18]:
# ATE estimation using Nearest Neighbors matching
# 1. Treated → Control
nn = NearestNeighbors(n_neighbors=1)
nn.fit(X_control)
_, control_indices = nn.kneighbors(X_treated)
treated_to_control = treated["Y"].values - control.iloc[control_indices.flatten()]["Y"].values

# 2. Control → Treated
nn.fit(X_treated)
_, treated_indices = nn.kneighbors(X_control)
control_to_treated = treated.iloc[treated_indices.flatten()]["Y"].values - control["Y"].values

# 3. Combine both directions
ate = np.mean(np.concatenate([treated_to_control, control_to_treated]))

print("Estimated ATE:", round(ate, 2))


Estimated ATE: 1.7


In [19]:
# ATT estimation using Nearest Neighbors matching
nn = NearestNeighbors(n_neighbors=1)
nn.fit(X_control)

_, indices = nn.kneighbors(X_treated)
matched_control_outcomes = control.iloc[indices.flatten()]['Y'].values
treatment_effects = treated['Y'].values - matched_control_outcomes
att = np.mean(treatment_effects)
print(f'Estimated ATT: {att}')

Estimated ATT: 1.8464085071912946


In [20]:
# ATU estimation using Nearest Neighbors matching
nn = NearestNeighbors(n_neighbors=1)
nn.fit(X_treated)

_, indices = nn.kneighbors(X_control)
matched_treatment_outcomes = treated.iloc[indices.flatten()]['Y'].values
treatment_effects = matched_treatment_outcomes - control['Y'].values
atu = np.mean(treatment_effects)
print(f'Estimated ATU: {atu}')
print(f'Optimal Treatment Effect: {np.max(treatment_effects)}')

Estimated ATU: 1.5494765534751722
Optimal Treatment Effect: 2.172469885577008
